In [1]:
import torch.utils.data
from datasets.shapenet_data_pc import ShapeNet15kPointClouds

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def get_dataset(dataroot, npoints,category):
    tr_dataset = ShapeNet15kPointClouds(root_dir=dataroot,
        categories=category.split(','), split='train',
        tr_sample_size=npoints,
        te_sample_size=npoints,
        scale=1.,
        normalize_per_shape=False,
        normalize_std_per_axis=False,
        random_subsample=True)
    te_dataset = ShapeNet15kPointClouds(root_dir=dataroot,
        categories=category.split(','), split='val',
        tr_sample_size=npoints,
        te_sample_size=npoints,
        scale=1.,
        normalize_per_shape=False,
        normalize_std_per_axis=False,
        all_points_mean=tr_dataset.all_points_mean,
        all_points_std=tr_dataset.all_points_std,
    )
    return tr_dataset, te_dataset

def get_dataloader(train_dataset, test_dataset=None):

    # if opt.distribution_type == 'multi':
    #     train_sampler = torch.utils.data.distributed.DistributedSampler(
    #         train_dataset,
    #         num_replicas=opt.world_size,
    #         rank=opt.rank
    #     )
    #     if test_dataset is not None:
    #         test_sampler = torch.utils.data.distributed.DistributedSampler(
    #             test_dataset,
    #             num_replicas=opt.world_size,
    #             rank=opt.rank
    #         )
    #     else:
    #         test_sampler = None
    # else:
    train_sampler = None
    test_sampler = None

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1,sampler=train_sampler,
                                                   shuffle=train_sampler is None, num_workers=1, drop_last=True)

    if test_dataset is not None:
        test_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1,sampler=test_sampler,
                                                   shuffle=False, num_workers=1, drop_last=False)
    else:
        test_dataloader = None

    return train_dataloader, test_dataloader, train_sampler, test_sampler

In [3]:
dataroot = "/home/yifan/studium/3D_Completion/DiT-3D_2024AILab/datasets/data/ShapeNetCore.v2.PC15k"
npoints = 2048
category = "camera"

train_dataset, _ = get_dataset(dataroot, npoints, category)
dataloader, _, train_sampler, _ = get_dataloader(train_dataset, None)
print(len(dataloader))

Total number of data:79
Min number of points: (train)2048 (test)2048
Total number of data:11
Min number of points: (train)2048 (test)2048
79


In [1]:
import torch.utils.data
from datasets.shapenet_data_comletion import ShapeNet15kPointCloudsPart

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
def get_dataset(dataroot, npoints,category):
    tr_dataset = ShapeNet15kPointCloudsPart(root_dir=dataroot,
        categories=category.split(','), split='train',
        tr_sample_size=npoints,
        te_sample_size=npoints,
        scale=1.,
        normalize_per_shape=False,
        normalize_std_per_axis=False,
        random_subsample=True)
    
    return tr_dataset

def get_dataloader(train_dataset, test_dataset=None):

    train_sampler = None
    test_sampler = None

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1,sampler=train_sampler,
                                                   shuffle=train_sampler is None, num_workers=4, drop_last=True)

    if test_dataset is not None:
        test_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1,sampler=test_sampler,
                                                   shuffle=False, num_workers=1, drop_last=False)
    else:
        test_dataloader = None

    return train_dataloader, test_dataloader, train_sampler, test_sampler

In [5]:
dataroot = "datasets/data/ShapeNetCompletion"
npoints = 2048
category = "chair"

train_dataset = get_dataset(dataroot, npoints, category)
dataloader, _, train_sampler, _ = get_dataloader(train_dataset, None)
print(len(dataloader))

all_mids:  5750


Processing directories: 100%|██████████| 5750/5750 [00:03<00:00, 1531.69it/s]


dataset complete pcd shape:  (16384, 3)
dataset partial pcd shape:  (512, 3)
all_points:  5750
all_part_ points:  5750
Total number of data:5750
Min number of points: (train)2048 (test)2048
Min number of parital points: (train)512 (test)512
5750


In [17]:
batch = next(iter(dataloader))
print(len(batch))
print(batch.keys())

12
dict_keys(['idx', 'train_points', 'train_partial_points', 'test_points', 'test_partial_points', 'mean', 'part_mean', 'std', 'part_std', 'cate_idx', 'sid', 'mid'])


In [7]:
print(batch['train_points'].shape) 

torch.Size([1, 2048, 3])


In [8]:
print(batch['train_partial_points'].shape) # batch, num_points, xyz

torch.Size([1, 512, 3])


In [9]:
import open3d as o3d
import numpy as np
                                                                                                                    


In [18]:
points = batch['train_partial_points'].squeeze(0).numpy()
points.shape

(512, 3)

In [19]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
o3d.visualization.draw_geometries([pcd])

In [20]:
full_points = batch['train_points'].squeeze(0).numpy()
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(full_points)
o3d.visualization.draw_geometries([pcd])

In [4]:
import torch
import torch.nn as nn

In [5]:
# 定义模型权重文件的路径
checkpoint_path = 'checkpoints/checkpoint.pth'

# 加载权重文件
checkpoint = torch.load(checkpoint_path)
print(checkpoint.keys())

# # 假设权重文件包含模型参数和训练状态（如 epoch）
# # 我们将 epoch 重置为 0
# if 'epoch' in checkpoint:
#     checkpoint['epoch'] = 0

# if 'optimizer_state_dict' in checkpoint:
#     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#     for param_group in optimizer.param_groups:
#         param_group['epoch'] = 0  # 注意：优化器本身没有 epoch 属性，但可以重置学习率等参数

# # 打印模型和优化器状态以验证加载是否正确
# print(model)
# print(optimizer)

# # （可选）保存更新后的权重文件
# torch.save(checkpoint, 'updated_model_checkpoint.pth')

dict_keys(['epoch', 'model_state', 'optimizer_state'])


In [6]:
checkpoint['epoch']

7649

In [3]:
checkpoint['optimizer_state']

{'state': {1: {'step': 2203200,
   'exp_avg': tensor([[[[[-1.3613e-05,  1.0674e-05, -3.1741e-06,  1.3334e-05],
              [ 5.2164e-06, -9.6691e-06,  1.0227e-05, -4.1681e-06],
              [ 1.3947e-05,  1.8611e-05, -1.3204e-06, -1.4381e-05],
              [-3.1704e-06,  4.0732e-06, -1.3089e-05, -1.9980e-05]],
   
             [[ 1.7781e-05,  9.1512e-06,  5.4825e-06,  3.1040e-06],
              [-9.3000e-07,  9.3341e-07,  4.6417e-06, -1.0485e-06],
              [-2.7089e-06,  1.4601e-05, -1.0469e-05, -7.9507e-06],
              [-8.1157e-06,  2.3711e-06, -9.6558e-06, -7.0553e-07]],
   
             [[ 2.0212e-05,  3.6234e-06,  9.7652e-06,  1.0489e-05],
              [ 1.5601e-05,  1.2437e-06, -1.2701e-05,  1.1597e-05],
              [-1.7667e-05,  2.0869e-05,  8.7536e-06, -4.6637e-06],
              [-2.2690e-07, -1.5180e-06, -7.4103e-06,  2.8713e-07]],
   
             [[ 5.9220e-06,  1.3561e-05, -1.3155e-05,  9.7232e-06],
              [ 2.0847e-06, -4.4112e-06,  1.0378e-05,  1.5